<h1>Roof Builder</h1>
<h5>Nicola Sardella - 440648 - workshop 4</h5>

<h2>Tetto 1 (semplice)</h2>
![alt text](tetto1.jpg)

<h2>Tetto 2 (complesso)</h2>
![alt text](tetto2.jpg)

In [1]:
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.00529521621563 seconds


In [2]:
vertici=[[0,0,0],[15,0,0],[15,10,0],[0,10,0],[5,5,0],[10,5,0],[5,5,2.5],[10,5,2.5]]
celle=[[1,2,5,6,7,8],[2,3,6,8],[3,4,5,6,7,8],[1,4,5,7]]
tetto1=MKPOL([vertici,celle,None])

vertici2=[[0,0,0],[4,0,0],[4,6,0],[9,6,0],[9,10,0],[0,10,0],[7,8,0],[2,8,0],[2,2,0],[7,8,2.5],
          [2,8,2.5],[2,2,2.5]]
celle2=[[1,2,9,12],[2,3,8,9,12,11],[3,4,7,8,10,11],[4,5,7,10],[5,6,7,8,10,11],[1,6,8,9,12,11]]
tetto2=MKPOL([vertici2,celle2,None])



<h1>variables:</h1>
skeleton --> SKEL_1 del tetto (travi del tetto)<br>
skeleton2 --> UKPOL dello SKEL_2 del tetto<br>
vertici--> vertici di skeleton2<br>
celle--> celle di skeleton2<br>
union--> un unico array nel quale si sostituiscono gli indici dei vertici con i loro valori reali<br>
vertex --> variabile di supporto per creare la lista di vertici aggiornata<br>
cells --> lista delle celle aggiornata<br>
vertex2 --> lista dei vertici aggiornata<br>
cover--> copertura del tetto<br>
struttura --> l'output della funzione ggpl_roof_builder(hpc)<br>

<h1>Geometric method:</h1>
La funzione prende in input dei valori hpc, rappresentativi di un modello grossolano di tetto. L'input è usato immediatamente per realizzare la struttura interna del tetto, tramite la funzione SKEL_1, ed applicandogli poi un offset di 0.2x0.2x0.2.
Come seconda operazione si esegue la funzione SKEL_2 sull'input e da questa vengono presi i valori dei vertici e delle celle convesse, tramite la funzione UKPOL.
Il risultato ottenuto è una grande quantità di valori, spesso ripetuti, all'interno della lista dei vertici. 
Di conseguenza, per risolvere il problema, ho operato in questo modo:
ho creato una pseudo-lista, uguale a quella delle celle, a differenza della quale, gli indici dei vertici sono sostituiti con i valori reali che essi assumono (x,y e z). Questa lista è rappresentata dalla variabile "union".
Dopodiché ho scandito la lista e ho preso una sola volta i valori contenuti in essa (eliminando i doppioni), creando una nuova lista chiamata "vertex".Poi ho riadattato e corretto i valori degli indici che identificavano i veritici all'interno della lista "cells".
In fine ho eliminato i vertici inutili dalla lista vertex (cioè quelli che corrispondevano alle proiezioni sul piano x-y dei vertici che avevano una z diversa da 0). Ho creato dunque una nuova lista vertex2.
In conclusione ho creato la copertura del tetto sfruttando le liste vertex2 e cells in questo modo: cover=MKPOL([vertex2,cells,None])
ed ho unito cover con la struttura interna del tetto generando così l'output della funzione.

In [32]:
def ggpl_roof_builder(hpc):
    """this function take as input some HTC values, rapresentative of a rough model of a roof.
    Its returns some HTC values, representative of a detailed model of a roof, with internal 
    structure (beams) and top cover"""
    skeleton = SKEL_1(hpc)
    skeleton = OFFSET([0.2,0.2,0.2])(skeleton)
    
  
    skeleton2 =UKPOL(SKEL_2(hpc))

    
    vertici=skeleton2[0]
    celle=skeleton2[1]
    union=[]
    vertex=[]
    cells=[]
    vertex2=[]
    
    for c in celle:
        u=[]
        for v in c:
            u.append(vertici[v-1])
        union.append(u)
    
    for u in union:
        for v in u:
            if v not in vertex:
                vertex.append(v)
            else:
                v=vertex.index(v)
    
    for u in union:
        a=[]
        for v in u:
            a.append(vertex.index(v)+1)
        cells.append(a)

    appeso=0
    for v in vertex:
        if v[2]>0:
            vertex2.append(v)
        else:
            for v2 in vertex:
                if v2[2]>0:
                    if v2[0]==v[0] and v2[1]==v[1]:
                        vertex2.append(v2)
                        appeso=1
            if not(appeso):
                vertex2.append(v)
            appeso=0
    cover=MKPOL([vertex2,cells,None])
    cover=STRUCT([T(3)(0.2),cover])
    cover=OFFSET([0.2,0.2,0.2])(cover)
    struttura= STRUCT([cover,skeleton])            

    return struttura

  
    
    
    
             
   
    

In [34]:
VIEW(ggpl_roof_builder(tetto1))


<h1>Results</h1>
<h2>Tetto 1:</h2>
![alt text](tetto1_1.PNG)
![alt text](tetto1_2.PNG)
<h2>Tetto 2:</h2>
![alt text](tetto2_1.PNG)
![alt text](tetto2_2.PNG)